## What was discovered?
* is_tourney (remark when game was in regular or tourney phase) variable doesnt matter according to feature importance and also we will predict only tourney phase games.
* to addres the problem of data leakage I removed scores from games (before game we dont know which teams shoot how many points) and also for every team i retrieve seed values. But if it is regular season i only have information about previous ones. So for regular season I retrieve seed from previous season and for tourney phase i retrieve information from current season.

## Load libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from custom_utils import transform_data, enrich_data, feature_importance
from IPython.display import display

## Load and prepare data

In [2]:
wregularseason = pd.read_csv("data/WRegularSeasonCompactResults.csv")
wtourneyseason = pd.read_csv("data/WNCAATourneyCompactResults.csv")

mregularseason = pd.read_csv("data/MRegularSeasonCompactResults.csv")
mtourneyseason = pd.read_csv("data/MNCAATourneyCompactResults.csv")

wprep = pd.concat([transform_data(wregularseason), transform_data(wtourneyseason)], axis=0).reset_index(drop=True)
mprep = pd.concat([transform_data(mregularseason), transform_data(mtourneyseason)], axis=0).reset_index(drop=True)

In [3]:
mprep_enh = enrich_data(mprep, "M")
wprep_enh = enrich_data(wprep, "W")
display(wprep_enh.loc[wprep_enh["Season"] > 1998].head())
mprep_enh.loc[mprep_enh["Season"] > 1985].head()

,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,SeedDiff
3868,1999,18,3104,3322,1,0,2,16,-14
3869,1999,18,3110,3416,1,0,16,16,0
3870,1999,18,3112,3453,1,0,3,14,-11
3871,1999,18,3116,3390,1,0,9,1,8
3872,1999,18,3124,3419,1,0,16,16,0


,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,SeedDiff
3737,1986,24,1181,1249,1,0,3,16,-13
3738,1986,24,1412,1401,1,0,7,16,-9
3739,1986,25,1133,1260,1,0,16,4,12
3740,1986,25,1197,1126,1,0,16,16,0
3741,1986,25,1208,1418,1,0,6,16,-10


In [4]:
mprep_enh = mprep_enh.loc[mprep_enh["Season"] > 1985]
wprep_enh = wprep_enh.loc[wprep_enh["Season"] > 1998]

X_featm = mprep_enh.drop("Result", axis=1)
ym = mprep_enh.Result

X_featw = wprep_enh.drop("Result", axis=1)
yw = wprep_enh.Result

X_trainm, X_testm, y_trainm, y_testm = train_test_split(X_featm, ym, test_size=0.1)
X_trainw, X_testw, y_trainw, y_testw = train_test_split(X_featw, yw, test_size=0.1)

In [5]:
rf_clfm = RandomForestClassifier()
rf_clfm.fit(X_trainm, y_trainm)

rf_clfw = RandomForestClassifier()
_ = rf_clfw.fit(X_trainw, y_trainw)

In [6]:
pred_probsm = rf_clfm.predict_proba(X_testm)[:, 1]
pred_probsw = rf_clfw.predict_proba(X_testw)[:, 1]

print(f"Brier score for mens data: {np.round(brier_score_loss(y_testm, pred_probsm), 3).item()}")
print(f"Brier score for womens data: {np.round(brier_score_loss(y_testw, pred_probsw), 3).item()}")

Brier score for mens data: 0.231
Brier score for womens data: 0.221


In [7]:
print("For men's games")
display(feature_importance(rf_clfm, mprep_enh))

print("For women's games")
display(feature_importance(rf_clfw, wprep_enh))

For men's games


,Feature,Importance_clf,Importance_mi
0,Season,0.164000,0.000524
1,DayNum,0.243930,0.000000
2,WTeamID,0.250758,0.019884
3,LTeamID,0.252013,0.022336
4,is_tournament,0.000710,0.000000
5,SeedW,0.023109,0.056072
6,SeedL,0.021907,0.054115
7,SeedDiff,0.043573,0.032092


For women's games


,Feature,Importance_clf,Importance_mi
0,Season,0.117960,0.001075
1,DayNum,0.224359,0.000000
2,WTeamID,0.273665,0.027202
3,LTeamID,0.271753,0.030315
4,is_tournament,0.000631,0.000000
5,SeedW,0.028088,0.047386
6,SeedL,0.029564,0.047030
7,SeedDiff,0.053979,0.050631
